# Numba func - C Callback function

This tutorial presents, how to set up a simple pipeline, that rotate images using numba_func operator.
This operator is a way for invoking user defined python code using DALI pipeline for allocating resources.

Operator is depending on Numba support https://numba.pydata.org/ which translate python and numpy code into LLVM code
which allows to reduce python interpreter overhead or completely eliminate it.

## Step-by-Step Guide
1. Let's start by importing DALI and a handful of utils.

In [18]:
from numba import cfunc, types, carray

from nvidia.dali import pipeline_def
import nvidia.dali as dali
import nvidia.dali.fn as fn
import nvidia.dali.types as dali_types
import nvidia.dali.numba_func as dali_numba

image_dir = "../data/images"
max_batch_size = 8

2. Next, we need to implement setup function for our operator. In setup function you are allowed to set output shapes basing on input shapes as well as set output data type.

In [ ]:
@cfunc(x, nopython=True)
def rot_image_setup(out_shape_ptr, out1_ndim, out1_dtype_ptr, in_shape_ptr, in1_ndim, in1_dtype, num_samples):
    in_shape = carray(in_shape_ptr, (num_samples, out1_ndim))
    out_shape = carray(out_shape_ptr, (num_samples, in1_ndim))
    out_type = carray(out_dtype_ptr, 1)
    out_type[0] = in_dtype
    for i in range(num_samples):
        out_shape[i][0]= in_shape[i][1]
        out_shape[i][1] = in_shape[i][0]
        out_shape[i][2] = in_shape[i][2]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/errors.py", line 744, in new_error_context
    yield
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/typeinfer.py", line 566, in __call__
    self.resolve(typeinfer, typevars, fnty)
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/typeinfer.py", line 642, in resolve
    if not sig.return_type.is_precise():
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/types/npytypes.py", line 486, in is_precise
    return self.dtype.is_precise()
AttributeError: 'int' object has no attribute 'is_precise'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/typeinfer.py", line 154, in propagate
    constraint(typeinfer)
  File "/home/rmaj/.local/lib/python3.6/site-packages/numba/core/typeinfer.py", line 566, in __call__
    self.resolve(typ

To learn more about cfunc you should go straight to [numba documentation](https://numba.pydata.org/numba-doc/latest/user/cfunc.html).

`setup_fn_sig` provide a function signature. As an arguments it takes respectively number of outputs and inputs.
Definition of setup function looks as follows 
```
def setup_fn(out1_shape_ptr, out1_ndim, out1_dtype_ptr, out2_shape_ptr, ..., \
    in1_shape_ptr, in1_ndim, in1_dtype, in2_shape_ptr, ..., num_samples):
```
So firstly we have all arguments connected with outputs, then inputs arguments and at the end number of sumples in batch.